# Lab KNN

In [ ]:
import pandas as pd #panda dataframes
import numpy as np

Location = r'Tabla_impago_full.xlsx'

##### Importo tabla 

In [ ]:
Tabla1 = pd.read_excel(Location,sheet_name="Tabla_B")

##### Leo tabla ignoro los IDs

In [ ]:
data = Tabla1.iloc[:,2:]
data=data.fillna(data.mean())

data1=data.iloc[:,0:2]

##### Usando toda la data encontraré los vecinos más cercanos de cada fila (empresa)

In [ ]:
from sklearn.neighbors import NearestNeighbors

##### Encuentro los k=2 vecinos más cercanos para cada fila en la data

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(data1)

##### Sklearn me devuelve el índice de los k vecinos más cercanos y la distancia promedio

In [ ]:
distances, indices = nbrs.kneighbors(data1)

##### Además puedo pedir que me cree una matriz de adyacencia (vecinos más cercanos)

In [ ]:
matrizvecinos=nbrs.kneighbors_graph(data1).toarray()

### Ahora ocuparé esta estrategia de los k-vecinos más cercanos para crear un modelo predictor de clase

##### Separo la variable dependiente de las explicativas

In [ ]:
Xs=data.iloc[:,1:]
Y=data.iloc[:,0]

##### Separo mi base de datos en dos muestras, una muestra de entrenamiento y una de prueba

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xs, Y, test_size=0.5, random_state=42)

### El comando KDTree leerá la base de datos de entrenamiento, y está será utilizada como mi "modelo"
    Es decir cada vez que quiera predecir algo en la base de datos test, modeloknn buscará los más parecidos en la 
    base training y hará una proyección en base a eso

In [ ]:
from sklearn.neighbors import KDTree

##### Leo la base X_Train

In [ ]:
modeloknn = KDTree(X_train.values, leaf_size=2)


distancias= np.empty((0,2))
indices= np.empty((0,2))

##### Luego para cada fila en la base Train le voy buscando sus 2 vecinos más cercanos

In [ ]:
counter=0
for row in X_test.values:
    counter=counter+1
    print counter
    row.reshape(1, -1)
    dist, indx = modeloknn.query(row, k=2)     
    distancias=np.vstack([distancias,dist])
    indices=np.vstack([indices,indx])
    

### Sklearn tiene una librería que realiza este proceso de manera automática
    Es decir, me permite predecir la clase de un ejemplo usando como "modelo" los k-vecinos más cercanos
    de una base de entrenamiento.
    

##### Importaré la libreria de KNeighborsClassifier y las librerias que me permiten medir el ajuste del clasificador

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

##### Configuro el modelo

In [ ]:
clf_knn=KNeighborsClassifier(2)

##### Lo entreno en la base train

In [ ]:
clf_knn.fit(X_train,y_train)

##### Una vez entrenado primero lo ocupo para predecir en la base train

In [ ]:
y_pred_train=clf_knn.predict(X_train)

    En la base train, los valores de los k-vecinos más cercanos a una fila de esa misma base se ocupan para hacer la predicción.
    Una vez que ya tengo las predicciones puedo calcular matriz de confusión, accuracy, recall, precision, auc, etc.

In [ ]:
cm_train=confusion_matrix(y_train,y_pred_train)

acc_train=metrics.accuracy_score(y_train,y_pred_train)
accuracy_train=clf_knn.score(X_train,y_train)

##### Calculo recall, precision, auc, fpr, tpr, etc en ambas bases

In [ ]:
fpr_train, tpr_train, thresholds_train = metrics.roc_curve(y_train, y_pred_train, pos_label=1)
auc_train=metrics.auc(fpr_train, tpr_train)
recalls_train=recall_score(y_train, y_pred_train, average=None)
preciss_train=precision_score(y_train, y_pred_train, average=None)
f1sc_train=f1_score(y_train, y_pred_train, average=None)

### Ahora puedo usar el modelo para predecir en testing
    Cada vez que quiera predecir algo en la base de datos test, clf_knn buscará los más parecidos 
    en la base training y hará una proyección en base a eso.

In [ ]:
y_pred_test=clf_knn.predict(X_test)

##### Una vez que ya tengo las predicciones puedo calcular matriz de confusión, accuracy, recall, precision, auc, etc...

In [ ]:
cm_test=confusion_matrix(y_test,y_pred_test)

acc_test=metrics.accuracy_score(y_test,y_pred_test)
accuracy_test=clf_knn.score(X_test,y_test)

fpr_test, tpr_test, thresholds_test = metrics.roc_curve(y_test, y_pred_test, pos_label=1)
auc_test=metrics.auc(fpr_test, tpr_test)
recalls_test=recall_score(y_test, y_pred_test, average=None)
preciss_test=precision_score(y_test, y_pred_test, average=None)
f1sc_test=f1_score(y_test, y_pred_test, average=None)

### Finalmente haremos un proceso que usará dos modelos para predecir, uno con KNN y otro con regresión logística
    Los resultados de aplicar los modelos en la muestra test quedaran guardados en la variable "resultados_testing"

##### Configuro y parametrizo los modelos que voy a entrenar

In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

names = ["Nearest Neighbors", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(2),
    LogisticRegression()]

resultados_testing=[] #genero una matriz vacia donde guardare los scores de los modelos Recall, Precision, AUC, etc

for name, clf in zip(names, classifiers):
    clf.fit(X_train,y_train)
    score_testing = clf.score(X_test, y_test)      #calculo scores en testing
    pred_testing=clf.predict(X_test)               #predigo en base testing

    #calculo recall, precision, auc, fpr, tpr, etc en ambas bases, reporto en vivo
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred_testing, pos_label=1)

    recalls=recall_score(y_test, pred_testing, average=None)
    preciss=precision_score(y_test, pred_testing, average=None)
    f1sc=f1_score(y_test, pred_testing, average=None)
    auc1=metrics.auc(fpr, tpr)
    resultados_testing.append((name,score_testing,recalls,preciss,f1sc,auc1))
    colum = ['Name', 'Score', 'Recalls', 'Precision', 'f1sc', 'auc1']
    resultados_testing2 = pd.DataFrame(resultados_testing, columns = colum)
    
    print("Hit Rates/Confusion Matrices en muestra testing:")
    print( resultados_testing2) 
    resultados_testing=[]
    